In [1]:
import pandas as pd
import numpy as np

In [2]:
converters = {'num_documento_cliente': str,
             'tipo_documento_cliente': str,
             'codigo_tienda': str, 
             'total_compra': float,
             'tipo_tienda': str,
             'latitud_tienda': float,
             'longitud_tienda': float,
             'id_barrio': str,
             'nombre_barrio': str}

In [3]:
%cd ../../

C:\Users\Asus\Documents\proyectos\datawarehouse


In [9]:
data = pd.read_csv('data/dataset.csv', converters=converters)

In [10]:
data.columns

Index(['num_documento_cliente', 'tipo_documento_cliente', 'codigo_tienda',
       'total_compra', 'tipo_tienda', 'latitud_tienda', 'longitud_tienda',
       'id_barrio', 'nombre_barrio', 'fecha_compra'],
      dtype='object')

In [11]:
len(data)

94610

In [12]:
data.head()

,num_documento_cliente,tipo_documento_cliente,codigo_tienda,total_compra,tipo_tienda,latitud_tienda,longitud_tienda,id_barrio,nombre_barrio,fecha_compra
0,6861792605025121524,1,14812028,22218042.0,Tienda especializada,3.402118,-76.525681,737998244,Canaverales Los Samanes,2022-06-15 15:49:11
1,1552970090928921643,1,14388938,21583407.0,Tienda especializada,3.468765,-76.484936,737998896,Jorge Eliecer Gaitan,2022-10-02 22:14:54
2,-4075904842556875666,1,12370698,6185349.0,Tienda especializada,3.400743,-76.524839,737998244,Canaverales Los Samanes,2022-01-10 0:27:57
3,3539339627033221084,1,10313583,10313583.0,Tienda especializada,3.369544,-76.523664,738000400,Lili,2022-09-02 16:30:24
4,5595554501572542610,1,14358071,28716142.0,Tienda especializada,3.419046,-76.497730,737996734,El Poblado,2022-03-20 2:56:49


In [14]:
data['fecha_compra'] = pd.to_datetime(data['fecha_compra'], format='%Y-%m-%d %H:%M:%S')
data['year_compra']= data['fecha_compra'].dt.year
data['month_compra']= data['fecha_compra'].dt.month_name('es_ES.utf8')
data['day_compra']= data['fecha_compra'].dt.day_name('es_ES.utf-8')
data['fecha_compra'] = data['fecha_compra'].dt.date

In [ ]:
#data.to_csv('data/dataset.csv', sep=',', index=False, encoding='utf-8')

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94610 entries, 0 to 94609
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   num_documento_cliente   94610 non-null  object 
 1   tipo_documento_cliente  94610 non-null  object 
 2   codigo_tienda           94610 non-null  object 
 3   total_compra            94610 non-null  float64
 4   tipo_tienda             94610 non-null  object 
 5   latitud_tienda          94610 non-null  float64
 6   longitud_tienda         94610 non-null  float64
 7   id_barrio               94610 non-null  object 
 8   nombre_barrio           94610 non-null  object 
 9   fecha_compra            94610 non-null  object 
 10  year_compra             94610 non-null  int32  
 11  month_compra            94610 non-null  object 
 12  day_compra              94610 non-null  object 
dtypes: float64(3), int32(1), object(9)
memory usage: 9.0+ MB


In [22]:
for col in data.columns:
    print(f' Nombre columna: {col}, cantidad categorias {data[col].nunique()}')

 Nombre columna: num_documento_cliente, cantidad categorias 10000
 Nombre columna: tipo_documento_cliente, cantidad categorias 5
 Nombre columna: codigo_tienda, cantidad categorias 6014
 Nombre columna: total_compra, cantidad categorias 16873
 Nombre columna: tipo_tienda, cantidad categorias 6
 Nombre columna: latitud_tienda, cantidad categorias 4714
 Nombre columna: longitud_tienda, cantidad categorias 4639
 Nombre columna: id_barrio, cantidad categorias 277
 Nombre columna: nombre_barrio, cantidad categorias 277
 Nombre columna: fecha_compra, cantidad categorias 365
 Nombre columna: year_compra, cantidad categorias 1
 Nombre columna: month_compra, cantidad categorias 12
 Nombre columna: day_compra, cantidad categorias 7


In [23]:
## En este caso para aquellas columnas que el sus categorias son inferiores a 265 categorias
## en la construcción de datawarehouse podemos codificar el tipo de dato con bytedict

In [24]:
documentos = data.loc[:, ['num_documento_cliente', 'tipo_documento_cliente']]
documentos.drop_duplicates(inplace=True)

In [25]:
documentos[documentos.duplicated('num_documento_cliente', keep=False)].sort_values('num_documento_cliente')

,num_documento_cliente,tipo_documento_cliente
92780,-1027836485982139155,2
10005,-1027836485982139155,1
14756,-1038754016974251967,1
92654,-1038754016974251967,2
1861,-1059790981306370284,1
...,...,...
94247,966908569164685428,2
92639,981867279217205960,2
547,981867279217205960,1
92847,988313494016465029,2


In [ ]:
## Existe numeros de documentos con dos tipo de documentos diferentes por tanto asumiremos que un cliente unico
# esta definido por el documento y tipo de documento, por tanto se debe crear una primary key con 
## esta combinación

In [26]:
tiendas = data.loc[:, ['codigo_tienda', 'tipo_tienda', 'latitud_tienda', 'longitud_tienda']]
tiendas.drop_duplicates(inplace=True)
tiendas[tiendas.duplicated('codigo_tienda', keep=False)].sort_values('codigo_tienda')

,codigo_tienda,tipo_tienda,latitud_tienda,longitud_tienda
1662,745,Tienda Outlet,3.414325,-76.547986
1663,745,Supermercado,3.451204,-76.532942
1779,747,Tienda Outlet,3.485807,-76.516292
1780,747,Supermercado,3.451204,-76.532942


In [ ]:
### Vemos que las tiendas tienen que ser diferenciadas por dos o más parámetros
## como el tipo_tienda no es un buen identificador,
## se tomara como identificador unico el codido tienda y sus coordenadas
## Igual que en el caso anterior, se debe construir una primary key compuesta para 
## el caso de la dimensión de tiendas

In [29]:
barrios = data.loc[:, ['id_barrio', 'nombre_barrio']]
barrios.drop_duplicates(inplace=True)
barrios[barrios.duplicated('id_barrio', keep=False)].sort_values('id_barrio')

,id_barrio,nombre_barrio


In [ ]:
## en este caso no tenemos ningun problema con la primary key, por tanto en la tabla de dimensión de barrios no
## es necesario realizar ninguna transformación